In [1]:
from plotly.offline import init_notebook_mode
from plotly.offline import iplot
from paramnb import Widgets
from pprint import pprint
import quest
import plotly.graph_objs as go
import pandas as pd
import warnings
warnings.simplefilter('ignore')
init_notebook_mode(connected=True)

In [2]:
quest_providers = quest.api.get_providers()
pprint(quest_providers)
quest_publishers = quest.api.get_publishers()
quest_publishers

['cuahsi-hs', 'ncdc', 'noaa', 'usgs-ned', 'usgs-nlcd', 'usgs-nwis']


['pub://cuahsi-hs:hs_normal']

In [3]:
quest_services = quest.api.get_services()
pprint(quest_services)
print("\nNumber of services: ", len(quest_services))
print("The one we want: ", quest_services[14])

['svc://cuahsi-hs:hs_geo',
 'svc://ncdc:ghcn-daily',
 'svc://ncdc:gsod',
 'svc://noaa:coops-meteorological',
 'svc://noaa:coops-water',
 'svc://noaa:ndbc',
 'svc://usgs-ned:1-arc-second',
 'svc://usgs-ned:13-arc-second',
 'svc://usgs-ned:19-arc-second',
 'svc://usgs-ned:alaska-2-arc-second',
 'svc://usgs-nlcd:2001',
 'svc://usgs-nlcd:2006',
 'svc://usgs-nlcd:2011',
 'svc://usgs-nwis:dv',
 'svc://usgs-nwis:iv']

Number of services:  15
The one we want:  svc://usgs-nwis:iv


In [4]:
nwis_s_features = quest.api.get_features(quest_services[14], filters={'bbox': [-91, 32.3, -90.8, 32.34]})
# bbox ['long min', 'lat min', 'long max', 'lat max']
#print(nwis_s_features)
#print(nwis_s_features[1])
nwis_s_features

['svc://usgs-nwis:iv/07289000']

In [5]:
#quest.api.new_collection("nwis")
#quest.api.delete(quest.api.get_collections())
collection_feature = quest.api.add_features('nwis', nwis_s_features[0])
collection_feature

['f6f2320d98e9487caf42a75a2f769392']

In [6]:
download_options = quest.api.download_options(collection_feature, fmt='param')[collection_feature[0]]
download_options

NwisServiceIV(end=None, name='iv', parameter=None, period='P365D', start=None)

In [7]:
Widgets(download_options)

<IPython.core.display.Javascript object>

VBox(children=(HTML(value='\n        <style>\n          .ttip { position: relative; display: inline-block; }\n          .ttip .ttiptext { visibility: hidden; background-color: #F8F8F8; outline: #CCCCCC solid thin;\n             color: black; border-radius: 2px; padding: 2px; text-align: center;\n             position: absolute; left: 53%; top: 30px; box-shadow: 7px 7px 10px #DDDDDD;\n             z-index: 100; min-width: 100px; font-size: 80%}\n          .ttip:hover .ttiptext { visibility: visible; }\n          .widget-dropdown .dropdown-menu { width: 100% }\n          .widget-select-multiple select { min-height: 100px; min-width: 300px;}\n        </style>\n        <div class="ttip"><b>iv</b></div>'), HBox(children=(HTML(value='<div class="ttip" style="padding: 5px; width: 67px;\n                      text-align: right;">parameter<span class="ttiptext">parameter</span></div>'), Dropdown(options=OrderedDict([('gage_height', 'gage_height'), ('streamflow', 'streamflow'), ('water_temperature', 'water_temperature')]), value=None))), HBox(children=(HTML(value='<div class="ttip" style="padding: 5px; width: 67px;\n                      text-align: right;">start<span class="ttiptext">start date</span></div>'), DatePicker(value=None))), HBox(children=(HTML(value='<div class="ttip" style="padding: 5px; width: 67px;\n                      text-align: right;">end<span class="ttiptext">end date</span></div>'), DatePicker(value=None))), HBox(children=(HTML(value='<div class="ttip" style="padding: 5px; width: 67px;\n                      text-align: right;">period<span class="ttiptext">time period (e.g. P365D = 365 days or P4W = 4 weeks)</span></div>'), Text(value='P365D', continuous_update=False)))), layout=Layout(display='flex', flex_flow='column'))

In [12]:
options = {'parameter':download_options.parameter, 
           'start': download_options.start, 
           'end': download_options.end, 
           'period': download_options.period
          }

In [13]:
staged_id = quest.api.stage_for_download(collection_feature, options=options)
staged_id

['d1848220cd834dd1b462e247193ed779']

In [14]:
quest.api.download_datasets(staged_id)

{'d1848220cd834dd1b462e247193ed779': 'downloaded'}

In [15]:
df = quest.api.open_dataset(staged_id[0])
df.head()

,qualifiers,streamflow
datetime,,
2017-02-16 16:00:00,A,573000.0
2017-02-16 17:00:00,A,576000.0
2017-02-16 18:00:00,A,576000.0
2017-02-16 19:00:00,A,573000.0
2017-02-16 20:00:00,A,574000.0


In [16]:
data = [go.Scatter(
          x=df.index,
          y=df['streamflow'],
          fill='tozeroy'
    )]

iplot(data)

In [17]:
quest.api.get_filters(filters={'dataset': staged_id[0]})

Could not load 'raster-fill': No module named 'terrapin'
Could not load 'raster-flow-accumulation': No module named 'terrapin'
Redefining 'Å' (<class 'pint.definitions.UnitDefinition'>)
Could not load 'raster-watershed-delineation': No module named 'terrapin'
Could not load 'raster-watershed-snap-outlet': No module named 'terrapin'
Redefining 'Å' (<class 'pint.definitions.UnitDefinition'>)
Could not load 'raster-fill': No module named 'terrapin'
Could not load 'raster-flow-accumulation': No module named 'terrapin'
Could not load 'raster-watershed-delineation': No module named 'terrapin'
Could not load 'raster-watershed-snap-outlet': No module named 'terrapin'


['ts-flow-duration', 'ts-remove-outliers', 'ts-resample', 'ts-unit-conversion']

In [18]:
filter_name = 'ts-resample'
filter_options = quest.api.apply_filter_options(filter_name, fmt='param')

In [37]:
Widgets(filter_options)

<IPython.core.display.Javascript object>

VBox(children=(HTML(value='\n        <style>\n          .ttip { position: relative; display: inline-block; }\n          .ttip .ttiptext { visibility: hidden; background-color: #F8F8F8; outline: #CCCCCC solid thin;\n             color: black; border-radius: 2px; padding: 2px; text-align: center;\n             position: absolute; left: 53%; top: 30px; box-shadow: 7px 7px 10px #DDDDDD;\n             z-index: 100; min-width: 100px; font-size: 80%}\n          .ttip:hover .ttiptext { visibility: visible; }\n          .widget-dropdown .dropdown-menu { width: 100% }\n          .widget-select-multiple select { min-height: 100px; min-width: 300px;}\n        </style>\n        <div class="ttip"><b>ts-resample</b></div>'), HBox(children=(HTML(value='<div class="ttip" style="padding: 5px; width: 60px;\n                      text-align: right;">dataset<span class="ttiptext">Dataset to apply filter to.</span></div>'), Dropdown(index=10, options=OrderedDict([('d12845b5f20741da865354a62ea78a5f', 'd12845b5f20741da865354a62ea78a5f'), ('d092adb6325943dba6ca2deb4435c317', 'd092adb6325943dba6ca2deb4435c317'), ('d5353fe88d2b4cabb1e2107e2ab63f76', 'd5353fe88d2b4cabb1e2107e2ab63f76'), ('da5d22f9e41e4b9eb285d2732ad374c8', 'da5d22f9e41e4b9eb285d2732ad374c8'), ('ded5bc8b777740a59abb80c29b2f2cdc', 'ded5bc8b777740a59abb80c29b2f2cdc'), ('d28a18b5d3974c2eac2d448d08550b51', 'd28a18b5d3974c2eac2d448d08550b51'), ('da337164e594428a8c716c631a87f165', 'da337164e594428a8c716c631a87f165'), ('d09005e8166d4e8d8871f8865adb2579', 'd09005e8166d4e8d8871f8865adb2579'), ('dfb96fcf62ad49579831ffb8cd240eb5', 'dfb96fcf62ad49579831ffb8cd240eb5'), ('d2e5c38e796b4c08b45cc3c4fb2ec71d', 'd2e5c38e796b4c08b45cc3c4fb2ec71d'), ('d1848220cd834dd1b462e247193ed779', 'd1848220cd834dd1b462e247193ed779'), ('de43c56025324b39b4709611a27f1f2c', 'de43c56025324b39b4709611a27f1f2c'), ('d0aaaa63860f48bc9ebe51195a1caa4a', 'd0aaaa63860f48bc9ebe51195a1caa4a')]), value='d1848220cd834dd1b462e247193ed779'))), HBox(children=(HTML(value='<div class="ttip" style="padding: 5px; width: 60px;\n                      text-align: right;">period<span class="ttiptext">resample frequency</span></div>'), Dropdown(index=1, options=OrderedDict([('daily', 'daily'), ('weekly', 'weekly'), ('monthly', 'monthly'), ('annual', 'annual')]), value='weekly'))), HBox(children=(HTML(value='<div class="ttip" style="padding: 5px; width: 60px;\n                      text-align: right;">method<span class="ttiptext">resample method</span></div>'), Dropdown(index=1, options=OrderedDict([('sum', 'sum'), ('mean', 'mean'), ('std', 'std'), ('max', 'max'), ('min', 'min'), ('median', 'median')]), value='mean')))), layout=Layout(display='flex', flex_flow='column'))

In [38]:
options = {'dataset':filter_options.dataset, 
           'period': filter_options.period, 
           'method': filter_options.method, 
          }

In [39]:
new_dataset = quest.api.apply_filter(filter_name, options=options)['datasets'][0]

In [74]:
df = quest.api.open_dataset(new_dataset)
date_list = []
for row in df.iterrows():
    date = str(row[0]).split("/")[0]
    date_list.append(date)
df['date'] = date_list
df.head()

<class 'pandas._libs.period.Period'>


,streamflow:weekly:mean
datetime,
2017-02-13/2017-02-19,578155.844156
2017-02-20/2017-02-26,547309.523810
2017-02-27/2017-03-05,475602.409639
2017-03-06/2017-03-12,602982.035928
2017-03-13/2017-03-19,857172.619048


In [73]:
data = [go.Scatter(
          x=df['date'],
          y=df['streamflow:weekly:mean'],
          fill='tozeroy'
    )]

iplot(data)

In [24]:
quest.api.authenticate_provider('cuahsi-hs')

Enter Username: AaronV77
Enter Password: ········


In [25]:
publish_options = quest.api.get_publish_options('pub://cuahsi-hs:hs_normal', fmt='param')['pub://cuahsi-hs:hs_normal']

In [26]:
Widgets(publish_options)

<IPython.core.display.Javascript object>

VBox(children=(HTML(value='\n        <style>\n          .ttip { position: relative; display: inline-block; }\n          .ttip .ttiptext { visibility: hidden; background-color: #F8F8F8; outline: #CCCCCC solid thin;\n             color: black; border-radius: 2px; padding: 2px; text-align: center;\n             position: absolute; left: 53%; top: 30px; box-shadow: 7px 7px 10px #DDDDDD;\n             z-index: 100; min-width: 100px; font-size: 80%}\n          .ttip:hover .ttiptext { visibility: visible; }\n          .widget-dropdown .dropdown-menu { width: 100% }\n          .widget-select-multiple select { min-height: 100px; min-width: 300px;}\n        </style>\n        <div class="ttip"><b>hs_normal</b></div>'), HBox(children=(HTML(value='<div class="ttip" style="padding: 5px; width: 60px;\n                      text-align: right;">title<span class="ttiptext">Title of resource</span></div>'), Text(value='example title', continuous_update=False))), HBox(children=(HTML(value='<div class="ttip" style="padding: 5px; width: 60px;\n                      text-align: right;">abstract<span class="ttiptext">An description of the resource to be added to HydroShare.</span></div>'), Text(value='example abstract', continuous_update=False))), HBox(children=(HTML(value='<div class="ttip" style="padding: 5px; width: 60px;\n                      text-align: right;">keywords<span class="ttiptext">list of keyword strings to describe the resource</span></div>'), Text(value='[]', continuous_update=False))), HBox(children=(HTML(value='<div class="ttip" style="padding: 5px; width: 60px;\n                      text-align: right;">dataset<span class="ttiptext">dataset to publish to HydroShare</span></div>'), Dropdown(options=OrderedDict([('d12845b5f20741da865354a62ea78a5f', 'd12845b5f20741da865354a62ea78a5f'), ('d092adb6325943dba6ca2deb4435c317', 'd092adb6325943dba6ca2deb4435c317'), ('d5353fe88d2b4cabb1e2107e2ab63f76', 'd5353fe88d2b4cabb1e2107e2ab63f76'), ('da5d22f9e41e4b9eb285d2732ad374c8', 'da5d22f9e41e4b9eb285d2732ad374c8'), ('ded5bc8b777740a59abb80c29b2f2cdc', 'ded5bc8b777740a59abb80c29b2f2cdc'), ('d28a18b5d3974c2eac2d448d08550b51', 'd28a18b5d3974c2eac2d448d08550b51'), ('da337164e594428a8c716c631a87f165', 'da337164e594428a8c716c631a87f165'), ('d09005e8166d4e8d8871f8865adb2579', 'd09005e8166d4e8d8871f8865adb2579'), ('dfb96fcf62ad49579831ffb8cd240eb5', 'dfb96fcf62ad49579831ffb8cd240eb5'), ('d2e5c38e796b4c08b45cc3c4fb2ec71d', 'd2e5c38e796b4c08b45cc3c4fb2ec71d'), ('d1848220cd834dd1b462e247193ed779', 'd1848220cd834dd1b462e247193ed779')]), value=None)))), layout=Layout(display='flex', flex_flow='column'))

In [32]:
new_publish_options = {'title': publish_options.title, 
                       'abstract': publish_options.abstract, 
                       'keywords': publish_options.keywords, 
                       'dataset': new_dataset}

In [33]:
quest.api.publish('pub://cuahsi-hs:hs_normal', options=new_publish_options)

Resource ID:  e68e4fc596374b3a9fdd7f985b1bb193
